## ENGRI 1120: Direct Calculation of the Total Gibbs Energy for Multiple Reactions

### Introduction

To solve this problem, we first need to setup the problem that we are trying to solve. The problem asks us to use the Direct Gibbs Energy Minimization (DGEM) approach. For multiple reactions, the Gibbs expression:

$$\hat{G} = \sum_{i\in\mathcal{M}}\bar{G}_{i}n_{i}$$

becomes:

$$\frac{1}{RT}\left(\hat{G}-\sum_{i\in\mathcal{M}}n_{i}^{\circ}G_{i}^{\circ}\right) = \sum_{j\in\mathcal{R}}\epsilon_{j}\left(\frac{\Delta{G}_{j}^{\circ}}{RT}\right) + \sum_{i\in\mathcal{M}}n_{i}\ln\hat{a}_{i}$$

where the number of mol for species _i_ is given by:

$$n_{i} = n_{i}^{\circ} + \sum_{r\in\mathcal{R}}\sigma_{ir}\epsilon_{r}$$. 

The quantity $\Delta{G}^{\circ}_{j}$ denotes the Gibbs energy of reaction for reaction _j_ (units: kJ/mmol), and $\hat{a}_{i}$ denotes the ratio of fugacity for component _i_, which (after the assumption of an ideal solution) becomes: 

$$\ln\hat{a}_{i} = \ln{x_{i}}$$ 

where $x_{i}$ denotes the mol fraction of component _i_. To estimate the equilibrium extent _vector_ we minimize Gibbs energy expression, subject to constraints. Our decision variables (what we are looking for) are the extents of reaction $\epsilon_{i},i=1,\dots,\mathcal{R}$. In this case the constraints are bounds on each extent $\epsilon_{i}\in\left[0,\star\right],\forall{i}$ and $n_{i}\geq{0},\forall{i}$.

#### Problem
Calculate the equilibrium extent of reaction and the equilibrium concentrations for the first five steps of glycolysis occuring in an  _E. coli_ MG-1655 cell free extract using Direct Gibbs Energy Minimization (DGEM).

The pathway we are exploring can be found [here](https://www.genome.jp/kegg-bin/show_pathway?eco00010).

__Assumption__
* The cell free extract has a constant V = 30.0μL
* The cell free extract acts like an _ideal_ liqud solution
* The cell free reaction is at a constant T, P
* The _default_ metabolic settings used by [eQuilibrator](https://equilibrator.weizmann.ac.il) are valid for this system

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-DirectGibbsEnergy-Multiple-Liq-Keq`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-DirectGibbsEnergy-Multiple-Liq-Keq/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-DirectGibbsEnergy-Multiple-Liq-Keq/Manifest.toml`


In [2]:
# load req packages -
using Optim
using PrettyTables

In [3]:
include("ENGRI-1120-Example-CodeLib.jl");

#### Constants

In [4]:
# conversion factor -
ΔG_sf = 1e9; # convert to mol to *mol

# set constants -
T = 37.0 + 273.15; # units: K
V = 30*(1/1e6); # units: L
R = 8.314*(1/1000)*(1/ΔG_sf); # units: kJ/nmol-K

# case flags -
simulate_with_sink_flag = false;

#### a) Setup the problem data

In [5]:
# setup problem parameters -
parameters_dict = Dict{String,Any}()

# conversion factor -
ΔG_sf = 1e9; # convert to mol to *mol

if (simulate_with_sink_flag == true)
    
    # we have an extra fake "sink" species -
    
    # what is my system dimensions?
    ℳ = 9 # number of metabolites
    ℛ = 6 # number of reactions

    # ΔG_formation data -
    G_formation_array = zeros(ℳ) 
    G_formation_array[1] = -409.4 		# 1 gluc kJ/mol
    G_formation_array[2] = -1304.7 		# 2 gluc-6-p kJ/mol
    G_formation_array[3] = -2280.7 		# 3 atp kJ/mol
    G_formation_array[4] = -1405.9 		# 4 adp kJ/mol
    G_formation_array[5] = -1302.1 		# 5 fruc-6-p kJ/mol
    G_formation_array[6] = -2193.6 		# 6 fruc-1,6-bis-p kJ/mol
    G_formation_array[7] = -1097.2 		# 7 dhap kJ/mol
    G_formation_array[8] = -1091.5 		# 8 ga3p kJ/mol
    G_formation_array[9] = -5000.0 	    # 9 sink
    parameters_dict["G_formation_array"] = (1/ΔG_sf)*G_formation_array # units: kJ/*mol

    # what are my initial condtions?
    n_initial_array = 1.0*ones(ℳ)
    n_initial_array[1] = 35.9*(V)*(1e9/1e3) 		# 1 gluc nmol
    n_initial_array[3] = 2000.0 					# 3 atp nmol
    parameters_dict["n_initial_array"] = n_initial_array

    # setup stoichiometric array -
    S = [

        # ϵ₁ ϵ₂  ϵ₃  ϵ₄  ϵ₅  ϵ₆
        -1.0 0.0 0.0 0.0 0.0 0.0 	; # 1 gluc
        1.0 -1.0 0.0 0.0 0.0 0.0	; # 2 gluc-6-p
        -1.0 0.0 -1.0 0.0 0.0 0.0 	; # 3 atp
        1.0 0.0 1.0 0.0 0.0 0.0     ; # 4 adp
        0.0 1.0 -1.0 0.0 0.0 0.0 	; # 5 fruc-6-p
        0.0 0.0 1.0 -1.0 0.0 0.0 	; # 6 fruc-1,6-bis-p
        0.0 0.0 0.0 1.0 1.0 0.0		; # 7 dhap
        0.0 0.0 0.0 1.0 -1.0 -1.0	; # 8 ga3p
        0.0 0.0 0.0 0.0 0.0 1.0 	; # 9 sink
    ];
    parameters_dict["S"] = S;
    
else
    
    # what is my system dimensions?
    ℳ = 8 # number of metabolites
    ℛ = 5 # number of reactions

    # ΔG_formation data -
    G_formation_array = zeros(ℳ) 
    G_formation_array[1] = -409.4 		# 1 gluc kJ/mol
    G_formation_array[2] = -1304.7 		# 2 gluc-6-p kJ/mol
    G_formation_array[3] = -2280.7 		# 3 atp kJ/mol
    G_formation_array[4] = -1405.9 		# 4 adp kJ/mol
    G_formation_array[5] = -1302.1 		# 5 fruc-6-p kJ/mol
    G_formation_array[6] = -2193.6 		# 6 fruc-1,6-bis-p kJ/mol
    G_formation_array[7] = -1097.2 		# 7 dhap kJ/mol
    G_formation_array[8] = -1091.5 		# 8 ga3p kJ/mol
    parameters_dict["G_formation_array"] = (1/ΔG_sf)*G_formation_array # units: kJ/*mol

    # what are my initial condtions?
    n_initial_array = 1.0*ones(ℳ)
    n_initial_array[1] = 35.9*(V)*(1e9/1e3) 		# 1 gluc nmol
    n_initial_array[3] = 2000.0 					# 3 atp nmol
    parameters_dict["n_initial_array"] = n_initial_array;

    # setup stoichiometric array -
    S = [

        # ϵ₁ ϵ₂  ϵ₃  ϵ₄  ϵ₅ 
        -1.0 0.0 0.0 0.0 0.0 	; # 1 gluc
        1.0 -1.0 0.0 0.0 0.0 	; # 2 gluc-6-p
        -1.0 0.0 -1.0 0.0 0.0 	; # 3 atp
        1.0 0.0 1.0 0.0 0.0 	; # 4 adp
        0.0 1.0 -1.0 0.0 0.0 	; # 5 fruc-6-p
        0.0 0.0 1.0 -1.0 0.0 	; # 6 fruc-1,6-bis-p
        0.0 0.0 0.0 1.0 1.0		; # 7 dhap
        0.0 0.0 0.0 1.0 -1.0	; # 8 ga3p
    ];
    
    parameters_dict["S"] = S;   
end

8×5 Matrix{Float64}:
 -1.0   0.0   0.0   0.0   0.0
  1.0  -1.0   0.0   0.0   0.0
 -1.0   0.0  -1.0   0.0   0.0
  1.0   0.0   1.0   0.0   0.0
  0.0   1.0  -1.0   0.0   0.0
  0.0   0.0   1.0  -1.0   0.0
  0.0   0.0   0.0   1.0   1.0
  0.0   0.0   0.0   1.0  -1.0

#### b) Optimization
Let's use the [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package to estimate the extent vector as a _constrained_ optimization problem 

In [6]:
# setup bounds -
L = zeros(ℛ)
U = maximum(n_initial_array)*ones(ℛ)


# Use the answer from Method 2 as a starting point 
xinitial = zeros(ℛ) .+ 0.001;
xinitial[1] = 0.5*maximum(U)

# setup the objective function -
OF(p) = objective_function(p, parameters_dict)

# call the optimizer -
opt_result = optimize(OF,L, U, xinitial, Fminbox(BFGS()))

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     -1.785180e+04

 * Found with
    Algorithm:     Fminbox with BFGS

 * Convergence measures
    |x - x'|               = 1.56e+03 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.24e-01 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 8.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    853
    ∇f(x) calls:   853


#### Build an extent of reaction table

In [7]:
ϵ = Optim.minimizer(opt_result)

# compute the dG_reaction -
G_formation_array = parameters_dict["G_formation_array"]
ΔG_rxn = transpose(S)*G_formation_array

# build a list of species -
if (simulate_with_sink_flag == true)
    # we need this for later -
    reaction_string_array = [
        "glc + atp = g6p + adp" 	;
        "g6p = f6p" 				;
        "f6p + atp = f16bp + adp" 	;
        "f16bp = dhap + ga3p" 		;
        "ga3p = dhap" 				;
        "sink"						;
    ];
else
    reaction_string_array = [
        "glc + atp = g6p + adp" 	;
        "g6p = f6p" 				;
        "f6p + atp = f16bp + adp" 	;
        "f16bp = dhap + ga3p" 		;
        "ga3p = dhap" 				;
    ];
end

# make the data table array -
data_table_array = Array{Any,2}(undef,ℛ,4)
for reaction_index = 1:ℛ
    data_table_array[reaction_index,1] = reaction_string_array[reaction_index]
    data_table_array[reaction_index,2] = ΔG_rxn[reaction_index]*(ΔG_sf)
    data_table_array[reaction_index,3] = ϵ[reaction_index]
    data_table_array[reaction_index,4] = ϵ[reaction_index]*(1/sum(n_initial_array))
end

# setup pretty table -
# header row -
path_table_header_row = (["Reaction","ΔG_rxn","ϵ", "ϵ_scaled"],["","kJ/nmol-K","nmol", "AU"]);

# write the table -
pretty_table(data_table_array; header=path_table_header_row)

┌─────────────────────────┬───────────┬─────────┬──────────┐
│                Reaction │    ΔG_rxn │       ϵ │ ϵ_scaled │
│                         │ kJ/nmol-K │    nmol │       AU │
├─────────────────────────┼───────────┼─────────┼──────────┤
│   glc + atp = g6p + adp │     -20.5 │  1072.4 │ 0.347842 │
│               g6p = f6p │       2.6 │  952.37 │  0.30891 │
│ f6p + atp = f16bp + adp │     -16.7 │ 909.215 │ 0.294912 │
│     f16bp = dhap + ga3p │       4.9 │ 644.181 │ 0.208946 │
│             ga3p = dhap │      -5.7 │  517.68 │ 0.167914 │
└─────────────────────────┴───────────┴─────────┴──────────┘


#### Build a concentration table

In [8]:
ϵ = Optim.minimizer(opt_result)
S = parameters_dict["S"]
n_initial_array = parameters_dict["n_initial_array"]
n = n_initial_array + S*ϵ

# setp table_data_array -
table_data_array = Array{Any,2}(undef,ℳ,5)

# build a list of species -
if (simulate_with_sink_flag == true)
    species_array = ["glucose","g6p","atp","adp","f6p","f16bp","dhap","ga3p", "sink"]
else
    species_array = ["glucose","g6p","atp","adp","f6p","f16bp","dhap","ga3p"]
end

# fill up the data table -
for species_index = 1:ℳ
    table_data_array[species_index,1] = species_array[species_index]
    table_data_array[species_index,2] = n_initial_array[species_index]
    table_data_array[species_index,3] = n[species_index]
    table_data_array[species_index,4] = (1/V)*n_initial_array[species_index]*(1e3)*(1/ΔG_sf) # converts to mM
    table_data_array[species_index,5] = (1/V)*n[species_index]*(1e3)*(1/ΔG_sf) # converts to mM
end

# setup pretty table -
# header row -
path_table_header_row = (["Species","n_i","n_f", "C_i", "C_f"],["","nmol","nmol", "mM", "mM"]);

# write the table -
pretty_table(table_data_array; header=path_table_header_row)

┌─────────┬────────┬─────────┬───────────┬──────────┐
│ Species │    n_i │     n_f │       C_i │      C_f │
│         │   nmol │    nmol │        mM │       mM │
├─────────┼────────┼─────────┼───────────┼──────────┤
│ glucose │ 1077.0 │ 4.60179 │      35.9 │ 0.153393 │
│     g6p │    1.0 │ 121.028 │ 0.0333333 │  4.03427 │
│     atp │ 2000.0 │ 18.3873 │   66.6667 │ 0.612909 │
│     adp │    1.0 │ 1982.61 │ 0.0333333 │  66.0871 │
│     f6p │    1.0 │ 44.1556 │ 0.0333333 │  1.47185 │
│   f16bp │    1.0 │ 266.033 │ 0.0333333 │  8.86778 │
│    dhap │    1.0 │ 1162.86 │ 0.0333333 │   38.762 │
│    ga3p │    1.0 │ 127.501 │ 0.0333333 │  4.25004 │
└─────────┴────────┴─────────┴───────────┴──────────┘
